# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2023-2024, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Maria de Neves de Fonseca (M.deNevesdeFonseca-1@student.tudelft.nl)
* (c) TU Delft, 2024

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions (also after the course is finished), and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for you that you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Task 3c - Generation of dataset with a simulated two-link robot (0p)
**Authors:** Giovanni Franzese (G.Franzese@tudelft.nl), Lorenzo Lyons (L.Lyons@tudelft.nl), Maximilian Stölzle (M.W.Stolzle@tudelft.nl)

In [ ]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

# import all Python modules
from distutils.util import strtobool
from functools import partial
from IPython.display import display, HTML  # For animations in the notebook
from jax import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
from jax import numpy as jnp
from pathlib import Path

from jax_double_pendulum.analysis import *
from jax_double_pendulum.dynamics import dynamical_matrices
from jax_double_pendulum.motion_planning import (
    generate_ellipse_trajectory,
    ELLIPSE_PARAMS,
)
from jax_double_pendulum.robot_parameters import ROBOT_PARAMS
from jax_double_pendulum.robot_simulation import simulate_robot
from jax_double_pendulum.visualization import animate_robot
import numpy as np
import os
import pandas as pd
import sys
from utils import process_data, split_2d_columns

# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))

# define folder where to save animations and plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)
# define path to datasets
datasets_dir = Path("datasets")
datasets_dir.mkdir(parents=True, exist_ok=True)

Import the PD feedback and the gravity compensation feedforward controller from Task 2a. **Make sure that you complete Task 2a.1 and 2a.2 first!**

In [ ]:
# import feedback and feedforward controller from problem_2/controllers.ipynb
sys.path.insert(0, Path.cwd().parent / "problem_2")
from ipynb.fs.full.controllers import (
    ctrl_fb_pd,
    ctrl_fb_pd_rel,
    ctrl_ff_gravity_compensation,
)

## Task 3c.1 - Dataset based on autonomous double pendulum initialized in downward position (0p)

In [ ]:
# file name to save the data
dataset_name = "trajectory_zero_gains_small_osc"

# simulation parameters
sim_duration = 15.0  # [s]
sim_dt = 1e-2  # [s]

# define time steps
t_ts = sim_dt * jnp.arange(int(sim_duration / sim_dt))

# initial state
th_0 = jnp.array([-jnp.pi / 2 + 0.1, -jnp.pi / 2 + 0.1])
th_d_0 = jnp.array([0.0, 0.0])

sim_ts = simulate_robot(
    rp=ROBOT_PARAMS,
    t_ts=t_ts,
    th_0=th_0,
    th_d_0=th_d_0,
)

# save the trajectory
# NOTE: we need to split the 2-column arrays in order to build a pandas data frame object
sim_ts_split_columns = split_2d_columns(sim_ts)

# convert data to a pandas dataframe
df = pd.DataFrame(sim_ts_split_columns)

# process the raw data
df = process_data(df)

# save the data
df.to_csv(datasets_dir / f"{dataset_name}.csv")

# produce robot animation
if not AUTOGRADING:
    print("producing animation")
    ani = animate_robot(
        ROBOT_PARAMS,
        sim_ts=sim_ts,
        step_skip=10,
        show=False,
        filepath=str(outputs_dir / f"task_3c-1_robot_animation_{dataset_name}.mp4"),
    )
    display(HTML(ani.to_html5_video()))

## Task 3c.2 - Dataset based on autonomous double pendulum initialized in upward position (0p)

In [ ]:
# file name to save the data
dataset_name = "trajectory_zero_gains_big_osc"

# simulation parameters
sim_duration = 50.0  # [s]
sim_dt = 1e-2  # [s]

# define time steps
t_ts = sim_dt * jnp.arange(int(sim_duration / sim_dt))

# initial state
th_0 = jnp.array([+0.5 * jnp.pi + 0.1, +0.5 * jnp.pi - 0.1])
th_d_0 = jnp.array([0.0, 0.0])

sim_ts = simulate_robot(
    rp=ROBOT_PARAMS,
    t_ts=t_ts,
    th_0=th_0,
    th_d_0=th_d_0,
)

# save the trajectory
# NOTE: we need to split the 2-column arrays in order to build a pandas data frame object
sim_ts_split_columns = split_2d_columns(sim_ts)

# convert data to a pandas dataframe
df = pd.DataFrame(sim_ts_split_columns)

# process the raw data
df = process_data(df)

# save the data
df.to_csv(datasets_dir / f"{dataset_name}.csv")

# produce robot animation
if not AUTOGRADING:
    print("producing animation")
    ani = animate_robot(
        ROBOT_PARAMS,
        sim_ts=sim_ts,
        step_skip=25,
        show=False,
        filepath=str(outputs_dir / f"task_3c-2_robot_animation_{dataset_name}.mp4"),
    )
    display(HTML(ani.to_html5_video()))

## Task 3c.3 - Dataset based on following a trajectory with high PD gains and with gravity compensation (0p)

In [ ]:
# file name to save the data
dataset_name = "trajectory_high_gains_grav_comp"

# simulation parameters
sim_duration = 15.0  # [s]
sim_dt = 1e-2  # [s]

# define time steps
t_ts = sim_dt * jnp.arange(int(sim_duration / sim_dt))

# generate reference trajectory
traj_ts = generate_ellipse_trajectory(
    rp=ROBOT_PARAMS,
    t_ts=t_ts,
    **ELLIPSE_PARAMS,
)

# set parameters according to the selected scenario
# fb controller parameters
kp = 2000 * jnp.eye(2)  # [Nm/rad]    1000
kd = 100 * jnp.eye(2)  # [Nm s/rad]  400

# initialize controllers
ctrl_ff = partial(
    ctrl_ff_gravity_compensation, partial(dynamical_matrices, ROBOT_PARAMS)
)
ctrl_fb = partial(ctrl_fb_pd_rel, kp=kp, kd=kd)

# initial state
th_0 = traj_ts["th_ts"][0]
th_d_0 = traj_ts["th_d_ts"][0]

sim_ts = simulate_robot(
    rp=ROBOT_PARAMS,
    t_ts=t_ts,
    th_0=th_0,
    th_d_0=th_d_0,
    th_des_ts=traj_ts["th_ts"],
    th_d_des_ts=traj_ts["th_d_ts"],
    th_dd_des_ts=traj_ts["th_dd_ts"],
    ctrl_ff=ctrl_ff,
    ctrl_fb=ctrl_fb,
)

# save the trajectory
# NOTE: we need to split the 2-column arrays in order to build a pandas data frame object
sim_ts_split_columns = split_2d_columns(sim_ts)

# add information about the reference trajectory
traj_ts_split_columns = split_2d_columns(traj_ts)

# convert data to a pandas dataframe
df = pd.DataFrame(sim_ts_split_columns)
df_reference_traj = pd.DataFrame(traj_ts_split_columns)

# add reference theta and theta dot to data
df["ref_th_d_1"] = df_reference_traj["th_d_ts_1"]
df["ref_th_d_2"] = df_reference_traj["th_d_ts_2"]
df["ref_th_1"] = df_reference_traj["th_ts_1"]
df["ref_th_2"] = df_reference_traj["th_ts_2"]

# process the raw data
df = process_data(df)

# save the data
df.to_csv(datasets_dir / f"{dataset_name}.csv")

# produce robot animation
if not AUTOGRADING:
    print("producing animation")
    ani = animate_robot(
        ROBOT_PARAMS,
        traj_ts=traj_ts,
        sim_ts=sim_ts,
        step_skip=10,
        show=False,
        filepath=str(outputs_dir / f"task_3c-3_robot_animation_{dataset_name}.mp4"),
    )
    display(HTML(ani.to_html5_video()))